In [1]:
from phi.torch.flow import *
import matplotlib.pyplot as plt

from src.eigenfluid import Eigenfluid

# Helper functions for visualization
from src.visu import *

import time

In [2]:
# Number of basis fields
N = 6**2 
# Domain sizes
DOMAIN = Box(x=math.PI, y=math.PI)
SAMPLING_SIZE = 32

start_time = time.time()
eigenfluid = Eigenfluid(N, DOMAIN, SAMPLING_SIZE)
end_time = time.time()
print("Eigenfluid initialized in {} seconds".format(end_time-start_time))

Eigenfluid initialized in 66.35890674591064 seconds


In [3]:
'''
velocity = eigenfluid.reconstruct_velocity()
smoke = CenteredGrid(Box(x=[1,2], y=[1,2]), 
                     extrapolation.BOUNDARY, 
                     x=100, y=100, 
                     bounds=DOMAIN)

vis.plot({
    "Initial Smoke": smoke,
    "Initial Velocity": get_rescaled_vel(velocity),
    "Initial Curl": field.curl(velocity),
    "Initial w (*100)": get_w_point_cloud(eigenfluid.w * 100)
}, same_scale = False)
'''

'\nvelocity = eigenfluid.reconstruct_velocity()\nsmoke = CenteredGrid(Box(x=[1,2], y=[1,2]), \n                     extrapolation.BOUNDARY, \n                     x=100, y=100, \n                     bounds=DOMAIN)\n\nvis.plot({\n    "Initial Smoke": smoke,\n    "Initial Velocity": get_rescaled_vel(velocity),\n    "Initial Curl": field.curl(velocity),\n    "Initial w (*100)": get_w_point_cloud(eigenfluid.w * 100)\n}, same_scale = False)\n'

## Traditional Fluid Simulation for Reference

In [4]:
# Initial values for reference simulation
velocity = eigenfluid.reconstruct_velocity()

trajectory_v = [velocity]
trajectory_curl = [field.curl(velocity)]

STEPS = 32
DT = 0.2

start_time = time.time()
for i in range(STEPS):
    velocity = advect.semi_lagrangian(velocity, velocity, dt=DT)
    velocity, _ = fluid.make_incompressible(velocity)

    trajectory_v.append(velocity)
    trajectory_curl.append(field.curl(velocity))
end_time = time.time()
print("Simulation finished in {} seconds".format(end_time-start_time))

trajectory_v_ref = math.stack(trajectory_v, batch('time'))
trajectory_curl_ref = math.stack(trajectory_curl, batch('time'))

Simulation finished in 5.617117881774902 seconds


In [5]:
print("Reference velocity field after {} timesteps: {}".format(STEPS, trajectory_v_ref.time[-1]))

Reference velocity field after 32 timesteps: CenteredGrid[(xˢ=32, yˢ=32, vectorᶜ=x,y), size=(x=3.142, y=3.142), extrapolation=boundary]


# Gradients from Eigenfluid simulation

In [6]:
def loss_function(w):
    eigenfluid.w = w
    velocities = [eigenfluid.reconstruct_velocity()]
    for t in range(STEPS):
        eigenfluid.w = eigenfluid.step_w_euler(eigenfluid.w, DT)
        velocities.append(eigenfluid.reconstruct_velocity())
    loss = field.l2_loss(velocities[-1] - trajectory_v_ref.time[-1])
    return loss, velocities

gradient_function = math.gradient(loss_function)

(loss, velocities), grad = gradient_function(eigenfluid.w)
print("Loss: {}".format(loss))

Loss: 1.4612883


In [7]:
curls = math.stack(list(map(field.curl, velocities)), batch('time'))
velocities = math.stack(velocities, batch('time'))

In [8]:
vis.plot({"Reference Velocity": trajectory_v_ref, 
          "Reference Curl": trajectory_curl_ref,
          "Eigenfluid Velocity (N={})\n Loss={:.2f} after {} steps".format(N, float(loss), STEPS): velocities, 
          "Eigenfluid Curl (N={})".format(N): curls
         }, 
         animate='time',
         same_scale=False)

<Figure size 640x480 with 0 Axes>